# Visualisation of OG1 recovered data from BioCarbon

In [2]:
import os
import sys
sys.path.append('C:/Users/flapet/OneDrive - NOC/Documents/utils_python')
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from functions.profiling import *
from tqdm import tqdm

In [3]:
current_dir = os.path.dirname(os.getcwd())

In [4]:
dataset_paths = os.listdir(current_dir + "/data/glider/raw/biocarbon_recovered_nc/")
dataset_full_paths = [os.path.join(current_dir, "data/glider/raw/biocarbon_recovered_nc", path) for path in dataset_paths]
varlist = ['temperature', 'salinity', 'chlorophyll', 'backscatter_700', 'oxygen_concentration', 'par']

In [ ]:
def plot_var(data, varname, savepath, ylim = 100):
    start_date = np.datetime64('2024-05-28T00:00:00.000000000')
    end_date = np.datetime64('2024-09-16T00:00:00.000000000')

    time = data['time'].values

    mask = (time > start_date) & (time < end_date)

    time2 = time[mask]
    depth = data['pressure'].values[mask]
    chl = data[varname].values[mask]

    if varname == 'par':
        mask_var = chl > 20
        time2 = time2[mask_var]
        depth = depth[mask_var]
        chl = chl[mask_var]
        ylim = 30
    
    if varname == 'salinity':
        mask_var = chl > 30
        time2 = time2[mask_var]
        depth = depth[mask_var]
        chl = chl[mask_var]


    plt.figure(figsize=(12,6))
    plt.scatter(time2,depth,c = chl)
    plt.ylim([0,ylim])
    plt.gca().invert_yaxis()
    plt.ylabel('Pressure (dbar)')
    plt.colorbar(label='{0} ({1})'.format(data[varname].long_name,data[varname].units))
    plt.title(label='{0} : {1}'.format(data.glider_name,data[varname].long_name))
    plt.savefig(savepath)  
    plt.close()

In [15]:
for path in tqdm(dataset_full_paths):
    dat = xr.open_dataset(path)
    glider = dat.glider_name
    if glider == 'Nelson' or glider == 'Churchill':
        savedir = current_dir + f'/output/plots/{glider}'
        if not os.path.isdir(savedir):
            os.mkdir(savedir)
        for var in tqdm(['par'], leave=False):
            print(f'Plotting {glider} {var} transect')
            savepath = savedir + f'/{glider}_{var}.png'
            plot_var(dat, var, savepath)
    dat.close()

    

 25%|██▌       | 1/4 [00:00<00:00,  3.00it/s]

Plotting Churchill par transect


 75%|███████▌  | 3/4 [00:04<00:01,  1.45s/it]

Plotting Nelson par transect


100%|██████████| 4/4 [00:07<00:00,  1.87s/it]


In [5]:
for path in tqdm(dataset_full_paths):
    dat = xr.open_dataset(path)
    glider = dat.glider_name
    savedir = current_dir + f'/output/plots/{glider}'
    if not os.path.isdir(savedir):
        os.mkdir(savedir)
    for var in tqdm(varlist, leave=False):
        print(f'Plotting {glider} {var} transect')
        savepath = savedir + f'/{glider}_{var}_1000.png'
        plot_var(dat, var, savepath, ylim = 1000)
    dat.close()

  0%|          | 0/4 [00:00<?, ?it/s]

Plotting Cabot temperature transect


Plotting Cabot salinity transect


Plotting Cabot chlorophyll transect


Plotting Cabot backscatter_700 transect


...

Plotting Nelson salinity transect


Plotting Nelson chlorophyll transect


Plotting Nelson backscatter_700 transect


Plotting Nelson oxygen_concentration transect


100%|██████████| 4/4 [1:12:16<00:00, 1084.09s/it]


In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(time2,depth2,c = chl2, vmax = 4)
plt.ylim([0,100])
plt.gca().invert_yaxis()
plt.ylabel('Pressure (dbar)')
plt.colorbar(label='{0} ({1})'.format(churchill['chlorophyll'].long_name,churchill['chlorophyll'].units))
plt.title(churchill['chlorophyll'].long_name);

In [ ]:
# Function to convert an array from NumPy datetime64 to Python float format
def datetime_to_float(dt):
  return (dt - np.datetime64('1900-01-01')) / np.timedelta64(1,'D')

# Function to convert an array from Python float to NumPy datetime64 format
def float_to_datetime(nums):
  return (nums * np.timedelta64(1,'D')) + np.datetime64('1900-01-01')

# Function to interpolate data from a specified float parameter to a uniform time and pressure grid
def interpolate_depth_section(data, param_name,specify_qc_flags=None,pres_interval=1.0):
  """
  Arguments:
      param_name: string with netCDF file parameter name (e.g., 'TEMP_ADJUSTED') to interpolate
      specify_qc_flags: None to ignore QC flags
                        or a list of QC flags (e.g., [1,2,3]) indicating which data to retain before interpolation
      pres_interval: vertical resolution for interpolating pressure (z) axis (default: 1.0 dbar)
  
  Returns:
      time_coord: 1-D NumPy array with original profile timestamps in np.datetime64 format
      pres_coord: 1-D NumPy array with a uniform pressure (z) coordinate from 0 dbar to the deepest recorded
                  pressure value, at a resolution of <pres_interval> dbar
      time_grid: 2-D NumPy array with the meshed grid of time_coord
      pres_grid: 2-D NumPy array with the meshed grid of pres_coord
      param_gridded: 2-D NumPy array with the interpolated parameter values at the locations of time_grid and pres_grid

  """

  # New grid points
  time_coord = data['time'].values
  pres_coord = np.arange(0,data['pressure'].max(),pres_interval)
  time_grid, pres_grid = np.meshgrid(time_coord,pres_coord)
  time_grid = datetime_to_float(time_grid)     # Convert from np.datetime64 to float

  # 1-D (flattened) versions of old grids and parameter values
  time_1D = data['time'].values.flatten()
  pres_1D = data['pressure'].values.flatten()
  param_1D = data[param_name].values.flatten()
  if param_1D.dtype == object:         # If parameter is an array of QC flag data
    param_1D = param_1D.astype(float)  # Convert QC flags from dtype 'object' to float
    interp_method = 'nearest'          # Use nearest-neighbor interpolation for QC flags to avoid unwanted averaging
  else:
    interp_method = 'linear'           # Use bilinear interpolation for normal data fields


  # Remove NaN values before interpolation
  time_1D = datetime_to_float(time_1D[~np.isnan(param_1D)])       # Convert from np.datetime64 to float
  pres_1D = pres_1D[~np.isnan(param_1D)]
  param_1D = param_1D[~np.isnan(param_1D)]

  # Interpolate from irregular points to grid
  param_gridded = interpolate.griddata((time_1D,pres_1D),param_1D,(time_grid,pres_grid),method=interp_method)

  # Return coordinates, grid, and gridded data
  return time_coord, pres_coord, float_to_datetime(time_grid), pres_grid, param_gridded

In [ ]:
# Interpolate data
param_name = 'chlorophyll'
time_coord, pres_coord, time_grid, pres_grid, param_gridded = interpolate_depth_section(churchill, param_name)

# Plot depth section of upper ocean only
plt.figure(figsize=(12,6))
plt.pcolormesh(time_grid,pres_grid,param_gridded)
plt.ylim([0,1000])
plt.gca().invert_yaxis()
plt.ylabel('Pressure (dbar)')
plt.colorbar(label='{0} ({1})'.format(data[param_name].long_name,data[param_name].units))
plt.title(data[param_name].long_name);

In [ ]:
def glider_to_pd(nc_path, glider_name, cols, PAR = True):
    dat = xr.open_dataset(nc_path)
    mission_date = pd.to_datetime("2024-08-15 18:57:00")
    dates_nc = dat['time']
    filtered_dat = dat.where(dates_nc > mission_date, drop=True)
    df_glid = filtered_dat[cols].to_dataframe()
    if PAR == True:
        if glider_name in ['Cabot', 'Doombar']:
            df_glid['DOWNWELLING_PAR'] = 'NaN'
        if glider_name in ['Churchill', 'Nelson']:
            df_glid['DOWNWELLING_PAR'] = pd.Series(filtered_dat['RAW_DOWNWELLING_PAR'].values)
    dat.close()
    df_glid['pressure'] = df_glid['pressure'].interpolate()
    #df_glid['CHLA'] = df_glid['CHLA'].interpolate()
    #df_glid['BBP700'] = df_glid['BBP700'].interpolate()
    df_glid['glider'] = glider_name
    return(df_glid)

In [ ]:
test = glider_to_pd("C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/glider/pomBODCREQ-5915/unit_398/nc_files/L0-timeseries/BioCarbon_churchill_398.nc", "Chruchill", ["time", "temperature", "pressure", "chlorophyll"])